# Getting started with DeepMatcher

Note: you can run **[this notebook live in Google Colab](https://colab.research.google.com/github/anhaidgroup/deepmatcher/blob/master/examples/getting_started.ipynb)** and use free GPUs provided by Google.

This tutorial describes how to effortlessly perform entity matching using deep neural networks. Specifically, we will see how to match pairs of tuples (also called data records or table rows) to determine if they refer to the same real world entity. To do so, we will need labeled examples as input, i.e., tuple pairs which have been annotated as matches or non-matches. This will be used to train our neural network using supervised learning. At the end of this tutorial, you will have a trained neural network as output which you can easily apply to unlabeled tuple pairs to make predictions.

In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

Mounted at /content/drive


As an overview, here are the 4 steps to use `deepmatcher` which we will go through in this tutorial:

<ol start="0">
  <li>Setup</li>
  <li>Process labeled data</li>
  <li>Define neural network model</li>
  <li>Train model</li>
  <li>Apply model to new data</li>
</ol>

Let's begin!

## Step 0. Setup

If you are running this notebook inside Colab, you will first need to install necessary packages by running the code below:

In [ ]:
try:
    import deepmatcher
except:
    !pip install -qqq deepmatcher

In [ ]:
# contorna erro legacy
!pip install torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Now let's import `deepmatcher` which will do all the heavy lifting to build and train neural network models for entity matching. 

In [ ]:
import pandas as pd
import deepmatcher as dm

We recommend having a GPU available for the training in Step 4. In case a GPU is not available, we will use all available CPU cores. You can run the following command to determine if a GPU is available and will be used for training:

In [ ]:
import torch
torch.cuda.is_available()

True

### Download sample data for entity matching

Now let's get some sample data to play with in this tutorial. We will need three sets of labeled data and one set of unlabeled data:

1. **Training Data:** This is used for training our neural network model.
2. **Validation Data:** This is used for determining the configuration (i.e., hyperparameters) of our model in such a way that the model does not overfit to the training set.
3. **Test Data:** This is used to estimate the performance of our trained model on unlabeled data.
4. **Unlabeled Data:** The trained model is applied on this data to obtain predictions, which can then be used for downstream tasks in practical application scenarios.

We download these four data sets to the `sample_data` directory:

In [ ]:
#!mkdir -p /content/drive/MyDrive/IC/unirTeste
#!wget -qnc -P /content/drive/MyDrive/IC/unirTeste/ http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured.zip

In [ ]:
#!unzip -q /content/drive/MyDrive/IC/unirTeste/Structured.zip -d /content/drive/MyDrive/IC/unirTeste

In [ ]:
#!ls /content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar

In [ ]:
'''
def merge_data(labeled, table_a, table_b, output):
  merged_csv = pd.read_csv(labeled).rename(columns={'ltable_id': 'left_id', 'rtable_id': 'right_id'})
  table_a_csv = pd.read_csv(table_a)
  table_a_csv = table_a_csv.rename(columns={col: 'left_' + col for col in table_a_csv.columns})
  table_b_csv = pd.read_csv(table_b)
  table_b_csv = table_b_csv.rename(columns={col: 'right_' + col for col in table_b_csv.columns})
  merged_csv = pd.merge(merged_csv, table_a_csv, on='left_id')
  merged_csv = pd.merge(merged_csv, table_b_csv, on='right_id')
  merged_csv['id'] = merged_csv[['left_id', 'right_id']].apply(lambda row: '_'.join([str(c) for c in row]), axis=1)
  del merged_csv['left_id']
  del merged_csv['right_id']
  merged_csv.to_csv(output, index=False)
'''

"\ndef merge_data(labeled, table_a, table_b, output):\n  merged_csv = pd.read_csv(labeled).rename(columns={'ltable_id': 'left_id', 'rtable_id': 'right_id'})\n  table_a_csv = pd.read_csv(table_a)\n  table_a_csv = table_a_csv.rename(columns={col: 'left_' + col for col in table_a_csv.columns})\n  table_b_csv = pd.read_csv(table_b)\n  table_b_csv = table_b_csv.rename(columns={col: 'right_' + col for col in table_b_csv.columns})\n  merged_csv = pd.merge(merged_csv, table_a_csv, on='left_id')\n  merged_csv = pd.merge(merged_csv, table_b_csv, on='right_id')\n  merged_csv['id'] = merged_csv[['left_id', 'right_id']].apply(lambda row: '_'.join([str(c) for c in row]), axis=1)\n  del merged_csv['left_id']\n  del merged_csv['right_id']\n  merged_csv.to_csv(output, index=False)\n"

In [ ]:
'''
merge_data(
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/train.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_train.csv')
merge_data(
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/valid.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_valid.csv')
merge_data(
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/test.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_test.csv')
'''

"\nmerge_data(\n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/train.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_train.csv')\nmerge_data(\n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/valid.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_valid.csv')\nmerge_data(\n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/test.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', \n    '/

To get an idea of how our data looks like, let's take a peek at the training dataset:

In [ ]:
#pd.read_csv('/content/drive/MyDrive/IC/ICReteste/Deepmatcher/TesteErros/TesteErrosLabel/Erros6_10/Datasets/Structured/DBLP-GoogleScholar/joined_train.csv').head()

## Step 1. Process labeled data

Before we can use our data for training, `deepmatcher` needs to first load and process it in order to prepare it for neural network training. Currently `deepmatcher` only supports processing CSV files. Each CSV file is assumed to have the following kinds of columns:

* **"Left" attributes (required):** Our goal is to match tuple pairs. "Left" attributes are columns that correspond to the "left" tuple or the first tuple in the tuple pair. These column names are expected to be prefixed with "left_" by default.
* **"Right" attributes (required):** "Right" attributes are columns that correspond to the "right" tuple or the second tuple in the tuple pair. These column names are expected to be prefixed with "right_" by default.
* **Label column (required for train, validation, test):** Column containing the labels (match or non-match) for each tuple pair. Expected to be named "label" by default
* **ID column (required):** Column containing a unique ID for each tuple pair. This is for evaluation convenience.  Expected to be named "id" by default.

More details on what data processing involves and ways to customize it are described in **[this notebook](https://nbviewer.jupyter.org/github/anhaidgroup/deepmatcher/blob/master/examples/data_processing.ipynb)**. 

### Processing labeled data
In order to process our train, validation and test CSV files we call `dm.data.process` in the following code snippet which will load and process the CSV files and return three processed `MatchingDataset` objects respectively. These dataset objects will later be used for training and evaluation. The basic parameters to `dm.data.process` are as follows:

* **path (required): ** The path where all data is stored. This includes train, validation and test. `deepmatcher` may create new files in this directory to store information about these data sets. This allows subsequent `dm.data.process` calls to be much faster.
* **train (required): ** File name of training data in `path` directory.
* **validation (required): ** File name of validation data in `path` directory.
* **test (optional): ** File name of test data in `path` directory.
* **ignore_columns (optional): ** Any columns in the CSV files that you may want to ignore for the purposes of training. These should be included here. 

Note that the train, validation and test CSVs must all share the same schema, i.e., they should have the same columns. Processing data involves several steps and can take several minutes to complete, especially if this is the first time you are running the `deepmatcher` package.

NOTE: If you are running this in Colab, you may get a message saying 'Memory usage is close to the limit.' You can safely ignore it for now. We are working on reducing the memory footprint.

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip --directory-prefix=/root/.vector_cache
!unzip /root/.vector_cache/wiki.en.zip -d /root/.vector_cache/
!rm /root/.vector_cache/wiki.en.vec

--2022-08-09 20:53:11--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10356881291 (9.6G) [application/zip]
Saving to: ‘/root/.vector_cache/wiki.en.zip’

wiki.en.zip         100%[===================>]   9.65G  16.8MB/s    in 10m 38s 

2022-08-09 21:03:50 (15.5 MB/s) - ‘/root/.vector_cache/wiki.en.zip’ saved [10356881291/10356881291]

Archive:  /root/.vector_cache/wiki.en.zip
  inflating: /root/.vector_cache/wiki.en.vec  
  inflating: /root/.vector_cache/wiki.en.bin  


In [ ]:
'''train, validation, test = dm.data.process(
    path='/content/drive/MyDrive/IC/ICReteste/Deepmatcher/TesteErros/TesteErrosLabel/Erros6_10/Datasets/Dirty/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test_error.csv')'''

"train, validation, test = dm.data.process(\n    path='/content/drive/MyDrive/IC/ICReteste/Deepmatcher/TesteErros/TesteErrosLabel/Erros6_10/Datasets/Dirty/DBLP-GoogleScholar/',\n    train='joined_train.csv',\n    validation='joined_valid.csv',\n    test='joined_test_error.csv')"

#### Peeking at processed data
Let's take a look at how the processed data looks like. To do this, we get the raw `pandas` table corresponding to the processed training dataset object. 

In [ ]:
'''train_table = train.get_raw_table()
train_table.head()'''

'train_table = train.get_raw_table()\ntrain_table.head()'

The processed attribute values have been tokenized and lowercased so they may not look exactly the same as the input training data. These modifications help the neural network generalize better, i.e., perform better on data not trained on. 

## Step 2. Define neural network model

In this step you tell `deepmatcher` what kind of neural network you would like to use for entity matching. The easiest way to do this is to use one of the several kinds of neural network models that comes built-in with `deepmatcher`. To use a built-in network, construct a `dm.MatchingModel` as follows:

`model = dm.MatchingModel(attr_summarizer='<TYPE>')`

where `<TYPE>` is one of `sif`, `rnn`, `attention` or `hybrid`. If you are not familiar with what these mean, we strongly recommend taking a look at either **[slides from our talk on deepmatcher](http://bit.do/deepmatcher-talk)** for a high level overview, or **[our paper](http://pages.cs.wisc.edu/~anhai/papers1/deepmatcher-sigmod18.pdf)** for a more detailed explanation. Here we give briefly describe the intuition behind these four model types:
* **sif:** This model considers the **words** present in each attribute value pair to determine a match or non-match. It does not take word order into account.
* **rnn:** This model considers the **sequences of words** present in each attribute value pair to determine a match or non-match.
* **attention:** This model considers the **alignment of words** present in each attribute value pair to determine a match or non-match. It does not take word order into account.
* **hybrid:** This model considers the **alignment of sequences of words** present in each attribute value pair to determine a match or non-match. This is the default.

`deepmatcher` is highly customizable and allows you to tune almost every aspect of the neural network model for your application scenario. **[This tutorial](https://nbviewer.jupyter.org/github/anhaidgroup/deepmatcher/blob/master/examples/matching_models.ipynb)** discusses the structure of `MatchingModel`s and how they can be customized.

For this tutorial, let's create a `hybrid` model for entity matching:

In [ ]:
'''model = dm.MatchingModel(attr_summarizer='attention')'''

"model = dm.MatchingModel(attr_summarizer='attention')"

## Step 3. Train model

Next, we train the defined neural network model using the processed training and validation data. To do so, we call the `run_train` method which takes the following basic parameters:

* **train:** The processed training dataset object (of type `MatchingDataset`).
* **validation:** The processed validation dataset object (of type `MatchingDataset`).
* **epochs:** Number of times to go over the entire `train` data for training the model.
* **batch_size:** Number of labeled examples (tuple pairs) to use for each training step. This value may be increased if you have a lot of training data and would like to speed up training. The optimal value is dataset dependent.
* **best_save_path:** Path to save the best model.
* **pos_neg_ratio**: The ratio of the weight of positive examples (matches) to weight of negative examples (non-matches). This value should be increased if you have fewer matches than non-matches in your data. The optimal value is dataset dependent.

Many other aspects of the training algorithm can be customized. For details on this, please refer the API documentation for **[run_train]()**

In [ ]:
'''model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)'''

"model.run_train(\n    train,\n    validation,\n    epochs=15,\n    batch_size=32,\n    best_save_path='attention_model.pth',\n    pos_neg_ratio=3)"

## Novos Testes

In [ ]:
!git clone https://github.com/pauloh48/IC.git

Cloning into 'IC'...
remote: Enumerating objects: 985, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 985 (delta 16), reused 7 (delta 0), pack-reused 930
Receiving objects: 100% (985/985), 252.10 MiB | 17.45 MiB/s, done.
Resolving deltas: 100% (600/600), done.
Checking out files: 100% (1274/1274), done.


### RNN

#### STRUCTURED

##### Amazon-Google

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 1762802
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:13


Finished Epoch 1 || Run Time:    7.6 | Load Time:    6.1 || F1:  27.96 | Prec:  37.03 | Rec:  22.46 || Ex/s: 501.68

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 15 || Run Time:    5.3 | Load Time:    4.4 || F1:  98.80 | Prec:  97.89 | Rec:  99.71 || Ex/s: 714.16

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 15 || Run Time:    0.8 | Load Time:    1.4 || F1:  58.69 | Prec:  65.10 | Rec:  53.42 || Ex/s: 1046.59

---------------------

Loading best model...
Training done.


tensor(60.1307, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 8


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 8 || Run Time:    0.8 | Load Time:    1.5 || F1:  61.17 | Prec:  62.11 | Rec:  60.26 || Ex/s: 999.18



tensor(61.1714, device='cuda:0')

##### Beer

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/Beer/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/Beer/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/Beer/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    0.3 | Load Time:    0.3 || F1:  11.43 | Prec:  13.33 | Rec:  10.00 || Ex/s: 421.21

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.0 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 580.89

* Best F1: tensor(0., device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 2 || Run Time:    0.3 | Load Time:    0.3 || F1:  36.73 | Prec: 100.00 | Rec:  22.50 || Ex/s: 440.89

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.0 | Load Time:    0.1 || F1:  23.53 | Prec:  66.67 | Rec:  14.29 || Ex/s: 581.98

* Best F1: tensor(23.5294, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 3


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    0.3 | Load Time:    0.3 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 436.02

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.0 | Load Time:    0.1 || F1:  52.00 | Prec:  36.11 | Rec:  92.86 || Ex/s: 620.30

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    0.3 | Load Time:    0.3 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 452.33

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.0 | Load Time:    0.1 || F1:  50.98 | Prec:  35.14 | Rec:  92.86 || Ex/s: 621.10

---------------------

Loading best model...
Training done.


tensor(59.4595, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 4


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 4 || Run Time:    0.0 | Load Time:    0.1 || F1:  66.67 | Prec:  50.00 | Rec: 100.00 || Ex/s: 601.01



tensor(66.6667, device='cuda:0')

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/DBLP-ACM/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/DBLP-ACM/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/DBLP-ACM/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    9.3 | Load Time:   12.1 || F1:  91.75 | Prec:  90.44 | Rec:  93.09 || Ex/s: 345.54

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    1.4 | Load Time:    3.8 || F1:  98.43 | Prec:  97.99 | Rec:  98.87 || Ex/s: 483.21

* Best F1: tensor(98.4305, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


Finished Epoch 15 || Run Time:    8.9 | Load Time:   11.9 || F1:  99.48 | Prec:  98.96 | Rec: 100.00 || Ex/s: 355.89

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    1.3 | Load Time:    3.8 || F1:  98.78 | Prec:  97.58 | Rec: 100.00 || Ex/s: 482.30

---------------------

Loading best model...
Training done.


tensor(98.8839, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 5 || Run Time:    1.3 | Load Time:    3.7 || F1:  98.44 | Prec:  97.36 | Rec:  99.55 || Ex/s: 486.43



tensor(98.4410, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/DBLP-GoogleScholar/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/DBLP-GoogleScholar/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/DBLP-GoogleScholar/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [#################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01

Computing principal components
0% [#################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:43


Finished Epoch 1 || Run Time:   20.0 | Load Time:   24.3 || F1:  83.07 | Prec:  78.69 | Rec:  87.96 || Ex/s: 388.60

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:11


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:43


Finished Epoch 15 || Run Time:   19.8 | Load Time:   24.4 || F1:  99.74 | Prec:  99.72 | Rec:  99.75 || Ex/s: 389.32

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 15 || Run Time:    3.0 | Load Time:    7.5 || F1:  92.97 | Prec:  92.03 | Rec:  93.93 || Ex/s: 549.35

---------------------

Loading best model...
Training done.


tensor(93.3949, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 6


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 6 || Run Time:    3.2 | Load Time:    7.6 || F1:  92.66 | Prec:  91.60 | Rec:  93.74 || Ex/s: 534.35



tensor(92.6559, device='cuda:0')

##### Walmart-Amazon

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 2576402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 1 || Run Time:    8.2 | Load Time:    6.7 || F1:  30.75 | Prec:  54.91 | Rec:  21.35 || Ex/s: 409.92

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 15 || Run Time:    7.8 | Load Time:    6.5 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 428.45

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 15 || Run Time:    1.2 | Load Time:    2.0 || F1:  68.64 | Prec:  80.00 | Rec:  60.10 || Ex/s: 629.56

---------------------

Loading best model...
Training done.


tensor(69.5906, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 13


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 13 || Run Time:    1.2 | Load Time:    2.1 || F1:  66.67 | Prec:  76.51 | Rec:  59.07 || Ex/s: 624.13



tensor(66.6667, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/AADatasets/Textual/Abt-Buy/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Textual/Abt-Buy/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Textual/Abt-Buy/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 1762802
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


Finished Epoch 1 || Run Time:    6.6 | Load Time:   10.5 || F1:  19.81 | Prec:  29.39 | Rec:  14.94 || Ex/s: 334.76

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:16


Finished Epoch 15 || Run Time:    6.3 | Load Time:   10.4 || F1:  99.11 | Prec:  98.56 | Rec:  99.68 || Ex/s: 344.43

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 15 || Run Time:    0.9 | Load Time:    3.1 || F1:  37.88 | Prec:  44.44 | Rec:  33.01 || Ex/s: 482.43

---------------------

Loading best model...
Training done.


tensor(44.6429, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 5 || Run Time:    0.9 | Load Time:    3.1 || F1:  41.19 | Prec:  38.96 | Rec:  43.69 || Ex/s: 481.05



tensor(41.1899, device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/AADatasets/Dirty/DBLP-ACM/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Dirty/DBLP-ACM/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Dirty/DBLP-ACM/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:25


Finished Epoch 1 || Run Time:   10.6 | Load Time:   15.5 || F1:  76.82 | Prec:  72.27 | Rec:  81.98 || Ex/s: 284.63

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:23


Finished Epoch 15 || Run Time:    9.4 | Load Time:   14.3 || F1:  99.66 | Prec:  99.70 | Rec:  99.62 || Ex/s: 313.28

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    1.4 | Load Time:    4.4 || F1:  94.64 | Prec:  91.93 | Rec:  97.52 || Ex/s: 432.68

---------------------

Loading best model...
Training done.


tensor(94.7368, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 7


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 7 || Run Time:    1.4 | Load Time:    4.3 || F1:  94.82 | Prec:  92.87 | Rec:  96.85 || Ex/s: 435.15



tensor(94.8181, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:51


Finished Epoch 1 || Run Time:   21.6 | Load Time:   30.2 || F1:  70.71 | Prec:  64.09 | Rec:  78.86 || Ex/s: 332.69

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:11


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:47


Finished Epoch 15 || Run Time:   20.0 | Load Time:   28.1 || F1:  99.83 | Prec:  99.84 | Rec:  99.81 || Ex/s: 357.58

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:11


Finished Epoch 15 || Run Time:    3.0 | Load Time:    8.6 || F1:  90.74 | Prec:  90.36 | Rec:  91.12 || Ex/s: 494.70

---------------------

Loading best model...
Training done.


tensor(90.9690, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 8


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 8 || Run Time:    3.5 | Load Time:    9.9 || F1:  89.49 | Prec:  91.50 | Rec:  87.57 || Ex/s: 426.29



tensor(89.4938, device='cuda:0')

### SIF

#### STRUCTURED

##### Amazon-Google

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/Amazon-Google/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/Amazon-Google/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/Amazon-Google/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 1 || Run Time:    3.3 | Load Time:    4.3 || F1:  24.56 | Prec:  39.18 | Rec:  17.88 || Ex/s: 906.94

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 15 || Run Time:    2.8 | Load Time:    4.1 || F1:  91.14 | Prec:  85.84 | Rec:  97.14 || Ex/s: 995.20

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 15 || Run Time:    0.6 | Load Time:    1.3 || F1:  46.91 | Prec:  55.56 | Rec:  40.60 || Ex/s: 1225.47

---------------------

Loading best model...
Training done.


tensor(52.2244, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 8


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 8 || Run Time:    0.6 | Load Time:    1.3 || F1:  55.94 | Prec:  50.69 | Rec:  62.39 || Ex/s: 1223.07



tensor(55.9387, device='cuda:0')

##### Beer

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/Beer/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/Beer/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/Beer/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    0.2 | Load Time:    0.3 || F1:  25.88 | Prec:  24.44 | Rec:  27.50 || Ex/s: 586.35

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.0 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 749.15

* Best F1: tensor(0., device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 2 || Run Time:    0.1 | Load Time:    0.3 || F1:   4.88 | Prec: 100.00 | Rec:   2.50 || Ex/s: 663.52

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.0 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 773.88

---------------------

===>  TRAIN Epoch 3


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    0.1 | Load Time:    0.3 || F1:  51.85 | Prec: 100.00 | Rec:  35.00 || Ex/s: 650.66

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.0 | Load Time:    0.1 || F1:  40.00 | Prec:  66.67 | Rec:  28.57 || Ex/s: 762.31

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    0.1 | Load Time:    0.3 || F1:  51.85 | Prec: 100.00 | Rec:  35.00 || Ex/s: 666.35

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.0 | Load Time:    0.1 || F1:  40.00 | Prec:  66.67 | Rec:  28.57 || Ex/s: 748.14

---------------------

Loading best model...
Training done.


tensor(42.1053, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Finished Epoch 9 || Run Time:    0.0 | Load Time:    0.1 || F1:  35.29 | Prec: 100.00 | Rec:  21.43 || Ex/s: 722.29



tensor(35.2941, device='cuda:0')

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/DBLP-ACM/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/DBLP-ACM/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/DBLP-ACM/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 1 || Run Time:    4.2 | Load Time:   10.9 || F1:  84.30 | Prec:  87.36 | Rec:  81.46 || Ex/s: 489.92

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 15 || Run Time:    3.8 | Load Time:   10.7 || F1:  99.48 | Prec:  98.96 | Rec: 100.00 || Ex/s: 511.81

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    0.8 | Load Time:    3.4 || F1:  98.54 | Prec:  98.00 | Rec:  99.10 || Ex/s: 593.89

---------------------

Loading best model...
Training done.


tensor(98.5475, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 8


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 8 || Run Time:    0.8 | Load Time:    3.3 || F1:  97.66 | Prec:  96.48 | Rec:  98.87 || Ex/s: 597.77



tensor(97.6641, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:33


Finished Epoch 1 || Run Time:    9.6 | Load Time:   23.8 || F1:  76.57 | Prec:  72.97 | Rec:  80.54 || Ex/s: 516.94

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:30


Finished Epoch 15 || Run Time:    8.4 | Load Time:   22.0 || F1:  99.30 | Prec:  98.98 | Rec:  99.63 || Ex/s: 566.23

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 15 || Run Time:    1.8 | Load Time:    6.7 || F1:  89.67 | Prec:  89.26 | Rec:  90.09 || Ex/s: 676.34

---------------------

Loading best model...
Training done.


tensor(89.9491, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 5 || Run Time:    1.9 | Load Time:    7.0 || F1:  89.38 | Prec:  89.50 | Rec:  89.25 || Ex/s: 647.94



tensor(89.3776, device='cuda:0')

##### Walmart-Amazon

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/Walmart-Amazon/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/Walmart-Amazon/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/AADatasets/Structured/Walmart-Amazon/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:13


Finished Epoch 1 || Run Time:    5.3 | Load Time:    8.0 || F1:  27.03 | Prec:  46.22 | Rec:  19.10 || Ex/s: 461.47

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 15 || Run Time:    3.7 | Load Time:    5.9 || F1:  99.05 | Prec:  98.62 | Rec:  99.48 || Ex/s: 643.10

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 15 || Run Time:    0.8 | Load Time:    1.9 || F1:  66.10 | Prec:  72.67 | Rec:  60.62 || Ex/s: 764.91

---------------------

Loading best model...
Training done.


tensor(67.2222, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 7


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 7 || Run Time:    0.8 | Load Time:    1.9 || F1:  61.54 | Prec:  68.35 | Rec:  55.96 || Ex/s: 771.54



tensor(61.5385, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:12


Finished Epoch 1 || Run Time:    2.5 | Load Time:    9.6 || F1:   0.92 | Prec:   7.69 | Rec:   0.49 || Ex/s: 475.97

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 1 || Run Time:    0.5 | Load Time:    2.8 || F1:   5.53 | Prec:  54.55 | Rec:   2.91 || Ex/s: 579.16

* Best F1: tensor(5.5300, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:11


Finished Epoch 15 || Run Time:    2.4 | Load Time:    9.4 || F1:  95.38 | Prec:  92.13 | Rec:  98.86 || Ex/s: 484.45

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 15 || Run Time:    0.5 | Load Time:    3.1 || F1:  31.91 | Prec:  35.29 | Rec:  29.13 || Ex/s: 534.51

---------------------

Loading best model...
Training done.


tensor(37.7682, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 4


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 4 || Run Time:    0.5 | Load Time:    2.9 || F1:  35.90 | Prec:  32.06 | Rec:  40.78 || Ex/s: 569.46



tensor(35.8974, device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


Finished Epoch 1 || Run Time:    4.8 | Load Time:   15.5 || F1:  58.49 | Prec:  63.21 | Rec:  54.43 || Ex/s: 366.31

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    0.8 | Load Time:    4.1 || F1:  68.56 | Prec:  56.60 | Rec:  86.94 || Ex/s: 506.06

* Best F1: tensor(68.5613, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


Finished Epoch 15 || Run Time:    3.9 | Load Time:   13.3 || F1:  98.15 | Prec:  96.86 | Rec:  99.47 || Ex/s: 429.47

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    0.8 | Load Time:    4.1 || F1:  82.98 | Prec:  81.98 | Rec:  84.01 || Ex/s: 509.31

---------------------

Loading best model...
Training done.


tensor(83.8780, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 7


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 7 || Run Time:    0.8 | Load Time:    4.0 || F1:  82.73 | Prec:  80.86 | Rec:  84.68 || Ex/s: 516.27



tensor(82.7283, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:37


Finished Epoch 1 || Run Time:    9.4 | Load Time:   28.2 || F1:  59.67 | Prec:  58.99 | Rec:  60.37 || Ex/s: 458.81

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:34


Finished Epoch 15 || Run Time:    8.6 | Load Time:   26.1 || F1:  98.78 | Prec:  98.94 | Rec:  98.63 || Ex/s: 496.48

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 15 || Run Time:    1.9 | Load Time:    8.2 || F1:  80.63 | Prec:  79.78 | Rec:  81.50 || Ex/s: 568.84

---------------------

Loading best model...
Training done.


tensor(80.8314, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 13


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 13 || Run Time:    1.9 | Load Time:    8.2 || F1:  79.12 | Prec:  78.90 | Rec:  79.35 || Ex/s: 567.32



tensor(79.1239, device='cuda:0')

### ATTENTION

#### STRUCTURED

##### Amazon-Google

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 3429602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   10.5 | Load Time:    4.1 || F1:  29.82 | Prec:  33.45 | Rec:  26.90 || Ex/s: 471.57

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 1 || Run Time:    2.0 | Load Time:    1.3 || F1:  40.16 | Prec:  38.58 | Rec:  41.88 || Ex/s: 694.47

* Best F1: tensor(40.1639, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 15 || Run Time:   10.5 | Load Time:    4.0 || F1:  93.64 | Prec:  88.83 | Rec:  99.00 || Ex/s: 473.91

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 15 || Run Time:    2.0 | Load Time:    1.3 || F1:  60.77 | Prec:  64.73 | Rec:  57.26 || Ex/s: 695.61

---------------------

Loading best model...
Training done.


tensor(63.5478, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 6


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 6 || Run Time:    2.0 | Load Time:    1.3 || F1:  62.16 | Prec:  56.69 | Rec:  68.80 || Ex/s: 683.98



tensor(62.1622, device='cuda:0')

##### Beer

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    0.6 | Load Time:    0.3 || F1:  23.33 | Prec:  35.00 | Rec:  17.50 || Ex/s: 310.25

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.1 | Load Time:    0.1 || F1:  32.56 | Prec:  19.44 | Rec: 100.00 || Ex/s: 449.61

* Best F1: tensor(32.5581, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 2 || Run Time:    0.8 | Load Time:    0.3 || F1:  42.70 | Prec:  38.78 | Rec:  47.50 || Ex/s: 246.74

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.1 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 444.18

---------------------

===>  TRAIN Epoch 3


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    0.6 | Load Time:    0.3 || F1:  90.91 | Prec:  83.33 | Rec: 100.00 || Ex/s: 321.70

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.1 | Load Time:    0.1 || F1:  64.71 | Prec:  55.00 | Rec:  78.57 || Ex/s: 478.77

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    0.6 | Load Time:    0.3 || F1:  93.02 | Prec:  86.96 | Rec: 100.00 || Ex/s: 320.54

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.1 | Load Time:    0.1 || F1:  64.71 | Prec:  55.00 | Rec:  78.57 || Ex/s: 469.10

---------------------

Loading best model...
Training done.


tensor(68.5714, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 12


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 12 || Run Time:    0.1 | Load Time:    0.1 || F1:  61.11 | Prec:  50.00 | Rec:  78.57 || Ex/s: 435.10



tensor(61.1111, device='cuda:0')

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   15.4 | Load Time:   10.4 || F1:  84.59 | Prec:  79.43 | Rec:  90.47 || Ex/s: 287.41

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 1 || Run Time:    3.0 | Load Time:    3.3 || F1:  93.67 | Prec:  88.10 | Rec: 100.00 || Ex/s: 388.58

* Best F1: tensor(93.6709, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:25


Finished Epoch 15 || Run Time:   15.4 | Load Time:   10.4 || F1:  99.40 | Prec:  98.81 | Rec: 100.00 || Ex/s: 287.68

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 15 || Run Time:    3.0 | Load Time:    3.3 || F1:  98.66 | Prec:  97.58 | Rec:  99.77 || Ex/s: 390.95

---------------------

Loading best model...
Training done.


tensor(98.6637, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 13


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 13 || Run Time:    3.1 | Load Time:    3.3 || F1:  98.32 | Prec:  97.56 | Rec:  99.10 || Ex/s: 386.99



tensor(98.3240, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   35.1 | Load Time:   21.8 || F1:  83.45 | Prec:  76.69 | Rec:  91.52 || Ex/s: 303.02

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:13


Finished Epoch 1 || Run Time:    6.9 | Load Time:    6.7 || F1:  91.04 | Prec:  86.66 | Rec:  95.89 || Ex/s: 421.10

* Best F1: tensor(91.0382, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:55


Finished Epoch 15 || Run Time:   34.7 | Load Time:   21.6 || F1:  99.61 | Prec:  99.41 | Rec:  99.81 || Ex/s: 306.08

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:13


Finished Epoch 15 || Run Time:    6.8 | Load Time:    6.7 || F1:  94.03 | Prec:  95.30 | Rec:  92.80 || Ex/s: 425.18

---------------------

Loading best model...
Training done.


tensor(94.4681, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 9 || Run Time:    6.8 | Load Time:    6.8 || F1:  93.30 | Prec:  93.60 | Rec:  92.99 || Ex/s: 422.12



tensor(93.2958, device='cuda:0')

##### Walmart-Amazon

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 5234402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   15.5 | Load Time:    6.1 || F1:  12.76 | Prec:  25.52 | Rec:   8.51 || Ex/s: 285.07

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    2.9 | Load Time:    1.9 || F1:  24.70 | Prec:  53.45 | Rec:  16.06 || Ex/s: 426.25

* Best F1: tensor(24.7012, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


Finished Epoch 15 || Run Time:   15.0 | Load Time:    5.9 || F1:  96.48 | Prec:  95.42 | Rec:  97.57 || Ex/s: 293.50

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    2.9 | Load Time:    1.9 || F1:  53.30 | Prec:  48.92 | Rec:  58.55 || Ex/s: 425.58

---------------------

Loading best model...
Training done.


tensor(54.2453, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 5 || Run Time:    3.0 | Load Time:    1.9 || F1:  52.26 | Prec:  50.73 | Rec:  53.89 || Ex/s: 420.53



tensor(52.2613, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 3429602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   10.6 | Load Time:    9.5 || F1:  19.66 | Prec:  23.84 | Rec:  16.72 || Ex/s: 285.45

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    1.9 | Load Time:    2.8 || F1:   7.14 | Prec:  44.44 | Rec:   3.88 || Ex/s: 406.04

* Best F1: tensor(7.1429, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:19


Finished Epoch 15 || Run Time:   10.4 | Load Time:    9.3 || F1:  92.04 | Prec:  88.59 | Rec:  95.78 || Ex/s: 290.92

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    2.1 | Load Time:    3.0 || F1:  50.57 | Prec:  60.96 | Rec:  43.20 || Ex/s: 378.93

---------------------

Loading best model...
Training done.


tensor(52.1739, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 6


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 6 || Run Time:    2.0 | Load Time:    2.8 || F1:  52.60 | Prec:  56.74 | Rec:  49.03 || Ex/s: 396.66



tensor(52.6042, device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   16.1 | Load Time:   12.6 || F1:  67.25 | Prec:  59.50 | Rec:  77.33 || Ex/s: 258.55

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 1 || Run Time:    3.3 | Load Time:    4.3 || F1:  89.82 | Prec:  85.69 | Rec:  94.37 || Ex/s: 327.35

* Best F1: tensor(89.8178, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:28


Finished Epoch 15 || Run Time:   16.0 | Load Time:   12.5 || F1:  98.55 | Prec:  97.92 | Rec:  99.17 || Ex/s: 260.46

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 15 || Run Time:    3.2 | Load Time:    4.0 || F1:  94.91 | Prec:  93.26 | Rec:  96.62 || Ex/s: 344.53

---------------------

Loading best model...
Training done.


tensor(95.1219, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 12


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 12 || Run Time:    3.2 | Load Time:    4.0 || F1:  95.81 | Prec:  93.75 | Rec:  97.97 || Ex/s: 343.47



tensor(95.8150, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   35.8 | Load Time:   25.5 || F1:  70.54 | Prec:  62.40 | Rec:  81.14 || Ex/s: 280.80

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 1 || Run Time:    7.1 | Load Time:    7.9 || F1:  81.17 | Prec:  78.02 | Rec:  84.58 || Ex/s: 382.69

* Best F1: tensor(81.1659, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:01


Finished Epoch 15 || Run Time:   36.2 | Load Time:   25.5 || F1:  95.19 | Prec:  93.24 | Rec:  97.22 || Ex/s: 279.05

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:15


Finished Epoch 15 || Run Time:    7.1 | Load Time:    7.9 || F1:  88.84 | Prec:  86.67 | Rec:  91.12 || Ex/s: 381.47

---------------------

Loading best model...
Training done.


tensor(89.0100, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 12


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 12 || Run Time:    7.2 | Load Time:    8.0 || F1:  88.74 | Prec:  86.64 | Rec:  90.93 || Ex/s: 377.46



tensor(88.7369, device='cuda:0')

### HYBRID

#### STRUCTURED

##### Amazon-Google

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 7133105
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:22


Finished Epoch 1 || Run Time:   18.3 | Load Time:    4.4 || F1:  30.87 | Prec:  37.92 | Rec:  26.04 || Ex/s: 302.91

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:21


Finished Epoch 15 || Run Time:   17.5 | Load Time:    4.3 || F1:  96.99 | Prec:  94.93 | Rec:  99.14 || Ex/s: 315.06

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    3.0 | Load Time:    1.4 || F1:  65.80 | Prec:  66.67 | Rec:  64.96 || Ex/s: 515.81

---------------------

Loading best model...
Training done.


tensor(69.5817, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 5 || Run Time:    3.1 | Load Time:    1.4 || F1:  68.94 | Prec:  61.90 | Rec:  77.78 || Ex/s: 510.74



tensor(68.9394, device='cuda:0')

##### Beer

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    1.0 | Load Time:    0.3 || F1:   2.78 | Prec:   3.12 | Rec:   2.50 || Ex/s: 206.79

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.2 | Load Time:    0.1 || F1:  38.30 | Prec:  27.27 | Rec:  64.29 || Ex/s: 364.99

* Best F1: tensor(38.2979, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 2 || Run Time:    1.0 | Load Time:    0.3 || F1:  26.42 | Prec:  53.85 | Rec:  17.50 || Ex/s: 210.85

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.2 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 361.15

---------------------

===>  TRAIN Epoch 3


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    1.0 | Load Time:    0.3 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 210.78

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.2 | Load Time:    0.1 || F1:  78.79 | Prec:  68.42 | Rec:  92.86 || Ex/s: 361.34

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    1.0 | Load Time:    0.3 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 211.40

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.2 | Load Time:    0.1 || F1:  78.79 | Prec:  68.42 | Rec:  92.86 || Ex/s: 350.09

---------------------

Loading best model...
Training done.


tensor(78.7879, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 9 || Run Time:    0.2 | Load Time:    0.1 || F1:  70.97 | Prec:  64.71 | Rec:  78.57 || Ex/s: 348.57



tensor(70.9677, device='cuda:0')

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   26.8 | Load Time:   10.7 || F1:  89.38 | Prec:  85.32 | Rec:  93.84 || Ex/s: 197.73

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 1 || Run Time:    4.8 | Load Time:    3.6 || F1:  96.00 | Prec:  92.31 | Rec: 100.00 || Ex/s: 295.59

* Best F1: tensor(96., device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:38


Finished Epoch 15 || Run Time:   27.7 | Load Time:   10.8 || F1:  99.40 | Prec:  98.81 | Rec: 100.00 || Ex/s: 192.86

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 15 || Run Time:    4.9 | Load Time:    3.6 || F1:  98.66 | Prec:  97.58 | Rec:  99.77 || Ex/s: 290.83

---------------------

Loading best model...
Training done.


tensor(98.7710, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 4


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 4 || Run Time:    5.3 | Load Time:    3.8 || F1:  98.76 | Prec:  98.87 | Rec:  98.65 || Ex/s: 272.31



tensor(98.7599, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   62.0 | Load Time:   22.1 || F1:  86.08 | Prec:  80.51 | Rec:  92.49 || Ex/s: 204.75

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:18


Finished Epoch 1 || Run Time:   11.0 | Load Time:    7.2 || F1:  91.66 | Prec:  85.63 | Rec:  98.60 || Ex/s: 315.93

* Best F1: tensor(91.6594, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:22


Finished Epoch 15 || Run Time:   61.8 | Load Time:   22.1 || F1:  99.75 | Prec:  99.63 | Rec:  99.88 || Ex/s: 205.40

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


Finished Epoch 15 || Run Time:   10.8 | Load Time:    7.2 || F1:  95.13 | Prec:  95.39 | Rec:  94.86 || Ex/s: 319.29

---------------------

Loading best model...
Training done.


tensor(95.1711, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 11


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 11 || Run Time:   11.0 | Load Time:    7.3 || F1:  94.90 | Prec:  94.20 | Rec:  95.61 || Ex/s: 313.82



tensor(94.8980, device='cuda:0')

##### Walmart-Amazon

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 11286907
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:31


Finished Epoch 1 || Run Time:   25.8 | Load Time:    6.1 || F1:  32.93 | Prec:  53.52 | Rec:  23.78 || Ex/s: 192.44

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:31


Finished Epoch 15 || Run Time:   26.0 | Load Time:    6.1 || F1:  99.05 | Prec:  98.62 | Rec:  99.48 || Ex/s: 191.48

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 15 || Run Time:    4.5 | Load Time:    2.0 || F1:  67.90 | Prec:  69.57 | Rec:  66.32 || Ex/s: 315.21

---------------------

Loading best model...
Training done.


tensor(68.7338, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 12


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 12 || Run Time:    4.6 | Load Time:    2.0 || F1:  63.66 | Prec:  65.22 | Rec:  62.18 || Ex/s: 309.15



tensor(63.6605, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 7133105
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   17.3 | Load Time:    9.5 || F1:  18.24 | Prec:  25.74 | Rec:  14.12 || Ex/s: 214.87

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 1 || Run Time:    2.9 | Load Time:    2.9 || F1:  20.27 | Prec:  33.33 | Rec:  14.56 || Ex/s: 327.71

* Best F1: tensor(20.2703, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:26


Finished Epoch 15 || Run Time:   17.5 | Load Time:    9.4 || F1:  98.16 | Prec:  96.55 | Rec:  99.84 || Ex/s: 213.52

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    3.0 | Load Time:    2.9 || F1:  66.32 | Prec:  70.49 | Rec:  62.62 || Ex/s: 324.77

---------------------

Loading best model...
Training done.


tensor(68.1319, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 6


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 6 || Run Time:    3.3 | Load Time:    3.0 || F1:  70.03 | Prec:  77.19 | Rec:  64.08 || Ex/s: 300.69



tensor(70.0265, device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   28.2 | Load Time:   13.0 || F1:  76.43 | Prec:  71.08 | Rec:  82.66 || Ex/s: 180.06

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 1 || Run Time:    5.2 | Load Time:    4.3 || F1:  90.91 | Prec:  83.97 | Rec:  99.10 || Ex/s: 259.96

* Best F1: tensor(90.9091, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:40


Finished Epoch 15 || Run Time:   28.2 | Load Time:   13.0 || F1:  99.44 | Prec:  98.96 | Rec:  99.92 || Ex/s: 180.06

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 15 || Run Time:    5.0 | Load Time:    4.3 || F1:  95.74 | Prec:  95.31 | Rec:  96.17 || Ex/s: 265.75

---------------------

Loading best model...
Training done.


tensor(96.4126, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 6


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 6 || Run Time:    5.2 | Load Time:    4.3 || F1:  97.20 | Prec:  96.66 | Rec:  97.75 || Ex/s: 260.18



tensor(97.2005, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/AADatasets/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   63.4 | Load Time:   26.1 || F1:  79.49 | Prec:  73.69 | Rec:  86.28 || Ex/s: 192.48

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:19


Finished Epoch 1 || Run Time:   11.4 | Load Time:    8.5 || F1:  85.34 | Prec:  77.13 | Rec:  95.51 || Ex/s: 289.26

* Best F1: tensor(85.3445, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:27


Finished Epoch 15 || Run Time:   62.8 | Load Time:   25.9 || F1:  99.58 | Prec:  99.50 | Rec:  99.66 || Ex/s: 194.08

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


Finished Epoch 15 || Run Time:   11.6 | Load Time:    8.5 || F1:  92.52 | Prec:  90.89 | Rec:  94.21 || Ex/s: 285.24

---------------------

Loading best model...
Training done.


tensor(92.7007, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 9 || Run Time:   11.8 | Load Time:    8.6 || F1:  92.22 | Prec:  90.84 | Rec:  93.64 || Ex/s: 281.19



tensor(92.2227, device='cuda:0')

## Step 4. Apply model to new data

### Evaluating on test data
Now that we have a trained model for entity matching, we can now evaluate its accuracy on test data, to estimate the performance of the model on unlabeled data.

In [ ]:
# Compute F1 on test set
'''model.run_eval(test)'''

### Evaluating on unlabeled data

We finally apply the trained model to unlabeled data to get predictions. To do this, we need to first process the unlabeled data.

#### Processing unlabeled data

To process unlabeled data, we use `dm.data.process_unlabeled`, as shown in the code snippet below. The basic parameters for this call are as follows:

* **path (required): ** The full path to the unlabeled data file (not just the directory).
* **trained_model (required): ** The trained model. The model is aware of the configuration of the training data on which it was trained, and so `deepmatcher` reuses the same configuration for the unlabeled data.
* **ignore_columns (optional): ** Any columns in the unlabeled CSV file that you may want to ignore for the purposes of evaluation. If not specified, the columns that were ignored while processing the training set will also be ignored while processing the unlabeled data.

Note that the unlabeled CSV file must have the same schema as the train, validation and test CSVs.

In [ ]:
'''
unlabeled = dm.data.process_unlabeled(
    path='sample_data/itunes-amazon/unlabeled.csv',
    trained_model=model)
'''

#### Obtaining predictions

Next, we call the `run_prediction` method which takes a processed data set object and returns a `pandas` dataframe containing tuple pair IDs (`id` column) and the corresponding match score predictions (`match_score` column). `match_scores` are in [0, 1] and a score above 0.5 indicates a match prediction.

In [ ]:
'''
predictions = model.run_prediction(unlabeled)
predictions.head()
'''

You may optionally set the `output_attributes` parameter to also include all attributes present in the original input table. As mentioned earlier, the processed attribute values will likely look a bit different from the attribute values in the input CSV files due to modifications such as tokenization and lowercasing.

In [ ]:
#predictions = model.run_prediction(unlabeled, output_attributes=True)
#predictions.head()

You can then save these predictions to CSV and use them for downstream tasks.

In [ ]:
#predictions.to_csv('sample_data/itunes-amazon/unlabeled_predictions.csv')

#### Getting predictions on labeled data

You can also get predictions for labeled data such as validation data. To do so, you can simply call the `run_prediction` method passing the validation data as argument.

In [ ]:
#valid_predictions = model.run_prediction(validation, output_attributes=True)
#valid_predictions.head()